In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import metrics
from keras.models import load_model

from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

Using TensorFlow backend.


Importando as top escolas predizidas pelos modelos criados nos noteboks passados

In [2]:
best_4pred = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred4(sem_ideb).csv')
best_3pred = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred3(sem_ideb).csv')
best_2pred = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred2(sem_ideb).csv')

best_4pred_bool = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred4(sem_ideb)_bool.csv')
best_3pred_bool = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred3(sem_ideb)_bool.csv')
best_2pred_bool = pd.read_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\best_esc_pred2(sem_ideb)_bool.csv')

Juntando os resultados dos modelos numéricos e booleanos

In [3]:
best_b = pd.concat([best_4pred_bool, best_3pred_bool, best_2pred_bool])
best_n = pd.concat([best_4pred, best_3pred, best_2pred])

C:\Users\Filipe Prates\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\Users\Filipe Prates\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Usando o value_counts podemos ver quais escolas mais apareceram nos top 1000 de cada modelo. O mais número não significa as escolas previstas para mais melhorar, e sim quais foram previstas para estar nas top1000 por mais modelos diferentes, assim podemos ter mais certeza que tais escolas realmente terão um resultado positivo no próximo Ideb

In [4]:
bestdf_b = pd.DataFrame()
bestdf_b = best_b['Cod_Escola'].value_counts()
bestdf_b

42020034    4
31194107    3
42116031    3
17039509    3
35901258    3
42018595    3
42091616    3
35031896    3
31033316    3
42101930    3
35913923    3
35090136    3
41093356    2
35282649    2
35022081    2
35921154    2
52030504    2
41002237    2
42064066    2
31174122    2
31269654    2
51011824    2
42109078    2
35198821    2
31098663    2
27044319    2
41029119    2
41130260    2
23229284    2
35904946    2
           ..
35037761    1
31226432    1
33088063    1
35210080    1
14320185    1
35902007    1
35039020    1
35234357    1
41024052    1
35072588    1
35035415    1
35027534    1
52079198    1
41385977    1
33026661    1
31181935    1
51050072    1
31024104    1
35234254    1
32018142    1
41134290    1
33059411    1
23206519    1
33063150    1
31219231    1
17001048    1
35919822    1
35211862    1
42143136    1
35229696    1
Name: Cod_Escola, Length: 2757, dtype: int64

In [5]:
bestdf_n = pd.DataFrame()
bestdf_n = best_n['Cod_Escola'].value_counts()
bestdf_n

29080657    6
23045442    6
33065250    5
22132481    5
42011337    4
13026577    4
23058331    4
12011410    4
21019649    4
31147940    4
31075337    4
23166371    3
29355028    3
23004584    3
11005971    3
31079855    3
31063495    3
25028022    3
15087042    3
42068541    3
23006200    3
31271926    3
23061944    3
33044473    3
31024163    3
31034410    3
31098256    3
33033200    3
15105415    3
23264306    3
           ..
31011843    1
29294061    1
29233755    1
41129458    1
33033765    1
33046077    1
35070919    1
26121786    1
31082040    1
42112109    1
31185043    1
33050163    1
43121608    1
43013678    1
29156904    1
41127463    1
51052067    1
32010524    1
28001826    1
26029944    1
31141399    1
29016657    1
41131533    1
15002586    1
33054215    1
33009155    1
27037185    1
41358848    1
15043550    1
28006330    1
Name: Cod_Escola, Length: 1689, dtype: int64

In [6]:
bestdf_all = pd.concat([best_4pred, best_3pred, best_2pred, best_4pred_bool, best_3pred_bool, best_2pred_bool])

C:\Users\Filipe Prates\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Exemplo de Ideb2017 e previsão de Ideb2019 de uma escola

In [9]:
a = bestdf_all.loc[bestdf_all['Cod_Escola']==23045442]
a

,Cod_Escola,Ideb2017,Unnamed: 0,dif,ensemble,ensemble_bool,pred_0709,pred_070911,pred_07091113,pred_07091113_bool,pred_0911,pred_091113,pred_09111315,pred_09111315_bool,pred_1113,pred_111315,pred_1315
304,23045442,6.1,1158,1.224584,7.324584,NaN,NaN,NaN,7.467259,NaN,NaN,NaN,7.181909,NaN,NaN,NaN,NaN
680,23045442,4.8,1159,1.070443,5.870443,NaN,NaN,NaN,5.952974,NaN,NaN,NaN,5.787912,NaN,NaN,NaN,NaN
97,23045442,6.1,16735,1.440404,7.540404,NaN,NaN,7.197933,NaN,NaN,NaN,7.754818,NaN,NaN,NaN,7.668460,NaN
200,23045442,4.8,16736,1.270884,6.070884,NaN,NaN,6.075233,NaN,NaN,NaN,6.094872,NaN,NaN,NaN,6.042548,NaN
109,23045442,4.8,12168,1.230060,6.030060,NaN,5.632533,NaN,NaN,NaN,5.939059,NaN,NaN,NaN,6.240522,NaN,6.308125
139,23045442,6.1,12167,1.180139,7.280139,NaN,6.831921,NaN,NaN,NaN,7.160983,NaN,NaN,NaN,7.601660,NaN,7.525990
407,23045442,6.1,1158,1.071956,NaN,7.171956,NaN,NaN,NaN,7.416303,NaN,NaN,NaN,6.927609,NaN,NaN,NaN
666,23045442,4.8,1159,1.008464,NaN,5.808464,NaN,NaN,NaN,6.106136,NaN,NaN,NaN,5.510792,NaN,NaN,NaN


In [10]:
bestdf_all.columns


Index(['Cod_Escola', 'Ideb2017', 'Unnamed: 0', 'dif', 'ensemble',
       'ensemble_bool', 'pred_0709', 'pred_070911', 'pred_07091113',
       'pred_07091113_bool', 'pred_0911', 'pred_091113', 'pred_09111315',
       'pred_09111315_bool', 'pred_1113', 'pred_111315', 'pred_1315'],
      dtype='object')

In [8]:
bestdf_all['Cod_Escola'].value_counts()

23045442    8
22132481    7
21019649    6
23058331    6
29080657    6
31217981    5
31194107    5
33065250    5
42030013    5
31307696    5
13026577    5
42011337    5
24039047    5
51056992    5
31191469    4
26135889    4
15033775    4
31272370    4
31174432    4
23069988    4
31129020    4
33093423    4
21022151    4
29282721    4
41049934    4
33005362    4
22110615    4
33040141    4
33084734    4
15015734    4
           ..
41037464    1
33017492    1
24050164    1
26042002    1
31226360    1
33046158    1
33050252    1
42094216    1
35278464    1
29413044    1
42089352    1
26129612    1
35920484    1
31260304    1
41030117    1
33068771    1
31003336    1
23084766    1
35279785    1
31111660    1
41372506    1
25007807    1
23169818    1
31108812    1
35219678    1
35086022    1
42018501    1
35901124    1
52054888    1
35207123    1
Name: Cod_Escola, Length: 3809, dtype: int64

Vendo as escolas que mais foram predizidas para melhorar nos diversos modelos

In [13]:
export = bestdf_all.drop(columns=['Unnamed: 0','pred_0709', 'pred_070911', 'pred_07091113',
       'pred_07091113_bool', 'pred_0911', 'pred_091113', 'pred_09111315',
       'pred_09111315_bool', 'pred_1113', 'pred_111315', 'pred_1315','ensemble_bool'])

In [16]:
export.to_csv

23045442    8
22132481    7
21019649    6
23058331    6
29080657    6
31217981    5
31194107    5
33065250    5
42030013    5
31307696    5
13026577    5
42011337    5
24039047    5
51056992    5
31191469    4
26135889    4
15033775    4
31272370    4
31174432    4
23069988    4
31129020    4
33093423    4
21022151    4
29282721    4
41049934    4
33005362    4
22110615    4
33040141    4
33084734    4
15015734    4
           ..
41037464    1
33017492    1
24050164    1
26042002    1
31226360    1
33046158    1
33050252    1
42094216    1
35278464    1
29413044    1
42089352    1
26129612    1
35920484    1
31260304    1
41030117    1
33068771    1
31003336    1
23084766    1
35279785    1
31111660    1
41372506    1
25007807    1
23169818    1
31108812    1
35219678    1
35086022    1
42018501    1
35901124    1
52054888    1
35207123    1
Name: Cod_Escola, Length: 3809, dtype: int64

In [34]:
a = export.loc[bestdf_all['Cod_Escola']==23045442]

In [35]:
b = export.loc[bestdf_all['Cod_Escola']==22132481]

In [36]:
c = export.loc[bestdf_all['Cod_Escola']==21019649]

In [37]:
d = export.loc[bestdf_all['Cod_Escola']==23058331]

In [38]:
e = export.loc[bestdf_all['Cod_Escola']==29080657]

In [39]:
f = export.loc[bestdf_all['Cod_Escola']==31217981]

In [40]:
g = export.loc[bestdf_all['Cod_Escola']==31194107 ]

In [41]:
h = export.loc[bestdf_all['Cod_Escola']==33065250]

In [42]:
i = export.loc[bestdf_all['Cod_Escola']==42030013]

In [43]:
j = export.loc[bestdf_all['Cod_Escola']==31307696]

In [44]:
top10 = pd.concat([a,b,c,d,e,f,g,h,i,j])

In [47]:
top10.dropna().to_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\top10.csv')

In [52]:
top10drop = top10.dropna()

In [56]:
top10drop.groupby('Cod_Escola').mean().

,Ideb2017,dif,ensemble
Cod_Escola,,,
21019649,4.85,1.557447,6.407447
22132481,2.84,1.129086,3.969086
23045442,5.45,1.236086,6.686086
23058331,4.25,1.038165,5.288165
29080657,2.50,1.106946,3.606946
31194107,6.10,1.397641,6.791391
31217981,4.00,1.010760,5.010760
31307696,4.90,1.639415,6.133165
33065250,6.80,1.615630,8.415630


In [54]:
top10drop.groupby('Ideb2017').mean()

,Cod_Escola,dif,ensemble
Ideb2017,,,
2.5,2.908066e+07,1.106946,3.606946
2.8,2.213248e+07,1.190403,3.990403
2.9,2.213248e+07,1.037110,3.937110
3.6,3.121798e+07,0.988778,4.588778
4.0,2.305833e+07,1.097933,5.097933
4.1,3.130770e+07,1.547823,5.647823
4.5,2.101965e+07,1.635044,6.135044
4.7,3.119411e+07,1.059897,5.759897
4.8,3.234919e+07,1.144032,5.944032


In [19]:
export.groupby('Cod_Escola').mean().to_csv(r'C:\Users\Filipe Prates\Documents\Projects\Datasets\BCG Challenge\original\best_schools\export.csv')

In [24]:
export.groupby('Cod_Escola').mean().sort_values(['dif'],ascending=False)

,Ideb2017,dif,ensemble
Cod_Escola,,,
41014375,5.2,5.668518,10.868518
33078238,5.8,4.787368,10.587368
50017802,4.4,4.555362,8.955362
29181429,5.7,3.684941,9.384941
41129636,7.3,2.761330,7.661330
35227407,7.4,2.756603,7.506603
35245948,7.2,2.629701,7.354701
41133331,7.3,2.613407,7.538407
26048841,2.6,2.605810,5.512588
